<a href="https://colab.research.google.com/github/Adamphoenix003/GNN-LinkPrediction/blob/main/Embedding_Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 27.0 MB/s eta 0:00:00


In [2]:
import networkx as nx
import numpy as np
import random
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression


In [4]:
G = nx.karate_club_graph()

# Create positive edge samples
edges = list(G.edges())
train_edges, test_edges = train_test_split(edges, test_size=0.3, random_state=42)

# Remove test edges from graph
G_train = G.copy()
G_train.remove_edges_from(test_edges)

# Generate negative samples
def generate_negative_edges(G, num_samples):
    neg_edges = []
    nodes = list(G.nodes())
    while len(neg_edges) < num_samples:
        u = random.choice(nodes)
        v = random.choice(nodes)
        if u != v and not G.has_edge(u, v):
            neg_edges.append((u, v))
    return neg_edges

neg_test_edges = generate_negative_edges(G, len(test_edges))


In [5]:
def generate_walks(G, num_walks=10, walk_length=20):
    walks = []
    nodes = list(G.nodes())
    for _ in range(num_walks):
        random.shuffle(nodes)
        for node in nodes:
            walk = [str(node)]
            current = node
            for _ in range(walk_length - 1):
                neighbors = list(G.neighbors(current))
                if neighbors:
                    current = random.choice(neighbors)
                    walk.append(str(current))
                else:
                    break
            walks.append(walk)
    return walks

walks = generate_walks(G_train)

deepwalk_model = Word2Vec(
    walks,
    vector_size=64,
    window=10,
    min_count=0,
    sg=1,
    epochs=10
)

deepwalk_embeddings = {int(node): deepwalk_model.wv[str(node)] for node in G.nodes()}


In [6]:
# Simple Node2Vec using biased random walks
def node2vec_walk(G, start_node, walk_length):
    walk = [start_node]
    for _ in range(walk_length - 1):
        neighbors = list(G.neighbors(walk[-1]))
        if neighbors:
            walk.append(random.choice(neighbors))
        else:
            break
    return walk

walks = []
for _ in range(10):
    for node in G_train.nodes():
        walk = node2vec_walk(G_train, node, 20)
        walks.append([str(n) for n in walk])

node2vec_model = Word2Vec(
    walks,
    vector_size=64,
    window=10,
    min_count=0,
    sg=1,
    epochs=10
)

node2vec_embeddings = {int(node): node2vec_model.wv[str(node)] for node in G.nodes()}



In [8]:
adj = nx.to_numpy_array(G_train)

svd = TruncatedSVD(n_components=34, random_state=42)
mf_embeddings = svd.fit_transform(adj)

mf_embeddings = {i: mf_embeddings[i] for i in range(len(mf_embeddings))}


In [9]:
def get_edge_features(embeddings, edge_list):
    return np.array([
        np.dot(embeddings[u], embeddings[v])
        for u, v in edge_list
    ])

def evaluate(embeddings):
    pos_scores = get_edge_features(embeddings, test_edges)
    neg_scores = get_edge_features(embeddings, neg_test_edges)

    y_true = np.concatenate([np.ones(len(pos_scores)), np.zeros(len(neg_scores))])
    y_scores = np.concatenate([pos_scores, neg_scores])

    return roc_auc_score(y_true, y_scores)


In [10]:
print("DeepWalk AUC:", evaluate(deepwalk_embeddings))
print("Node2Vec AUC:", evaluate(node2vec_embeddings))
print("Matrix Factorization AUC:", evaluate(mf_embeddings))


DeepWalk AUC: 0.6579861111111112
Node2Vec AUC: 0.6284722222222221
Matrix Factorization AUC: 0.5989583333333333
